In [ ]:
import os
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ.pop("HF_HUB_OFFLINE", None)

In [ ]:
pip install torch transformers datasets

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install --upgrade accelerate

In [ ]:
import accelerate
print(accelerate.__version__)

1.10.1


In [ ]:
import sys
print(sys.executable)

/usr/bin/python3


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2", force_download=True)
model = AutoModelForCausalLM.from_pretrained("gpt2", force_download=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving story.txt to story.txt


In [ ]:
# Fix padding issue for GPT-2
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [ ]:
from datasets import Dataset

# Read your story.txt into a list of dicts
with open("story.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Wrap in dataset format
dataset = Dataset.from_dict({"text": lines})

# Split into train/test (80-20 split for example)
dataset = dataset.train_test_split(test_size=0.2)


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

print(tokenized_dataset["train"][0])  # check tokenized output


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

{'input_ids': [45, 576, 264, 3949, 465, 8887, 11, 15950, 9480, 13, 564, 250, 18274, 353, 306, 13, 447, 251, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Data collator (handles padding & dynamic masking)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    prediction_loss_only=True,
    report_to="none",
    push_to_hub=False
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"], # Specify the training dataset
    data_collator=data_collator
)

# Start training
trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.113700
200,2.472300


TrainOutput(global_step=225, training_loss=2.7370486789279513, metrics={'train_runtime': 52.9225, 'train_samples_per_second': 17.006, 'train_steps_per_second': 4.252, 'total_flos': 58790707200000.0, 'train_loss': 2.7370486789279513, 'epoch': 3.0})

In [ ]:
# Save fine-tuned model and tokenizer
trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json',
 './gpt2-finetuned/tokenizer.json')

In [ ]:
from transformers import pipeline

# Load your fine-tuned model
generator = pipeline(
    "text-generation",
    model="./gpt2-finetuned",
    tokenizer=tokenizer
)

# Generate text
prompt = "Once upon a time"
output = generator(prompt, max_length=100, num_return_sequences=1)

print(output[0]["generated_text"])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upon a time they met in the café, Kazuma whispered a few words. He turned to her and asked, “What’s going on?”

She smiled warmly. “She said she just met your mother. And she said her mother was always polite.”

Kazuma smiled slightly. “She’s right.”

Kazuma’s mother.

Kazuma watched her mother, motherhood, and her quiet life with a warm warmth in his eyes. His mother loved making sure everyone was happy. She never gave up.

”She gave so much.”

Kazuma turned, and with a nod, he walked down the hall to his room. He pulled out a cup, and placed it between his lips.

”You’ve got a lot to think about.”

Kazuma gave her a quiet nod. “I’ll start by talking about your life.”

”I’ll start with your mother.”

Kazuma finished his cup, and returned to his room.

”And then,” he said quietly.


In [ ]:
output = generator(
    "Once upon a time, in a dark kingdom",
    max_new_tokens=200,
    temperature=0.9,
    top_p=0.9,
    repetition_penalty=1.2,
    num_return_sequences=1
)

print(output[0]["generated_text"])


Once upon a time, in a dark kingdom shrouded with magic and sinister spirits—Karma had taken the throne. In his stead lay Hina Solo…
The Queen of Blades — but for her own sakes! “Told us they lived under great illusions that their strength could ever be matched by any other single warrior on this earth’s battlefield. They tried to steal our secrets from everyone else because it was only right we were here so long as something truly powerful wasn't around: nothing more than beasts drawn out of dungeons or demon hunters called shadows like fireflies trapped within crevices between floors nearby. It didn?t work; no matter how many times she said cruel things about them once at first glance, all those years later now spoke plainly why people still feared even these kind creatures beyond redemption anymore... For everything you did before your death stood beside King Araragi's resolve towards justice." ―Hino stumbles over words carefully woven into every word he says while gently teasing Ka

In [ ]:
!zip -r gpt2-finetuned.zip ./gpt2-finetuned

  adding: gpt2-finetuned/ (stored 0%)
  adding: gpt2-finetuned/runs/ (stored 0%)
  adding: gpt2-finetuned/runs/Sep30_09-33-57_1f0c1ae7bca4/ (stored 0%)
  adding: gpt2-finetuned/runs/Sep30_09-33-57_1f0c1ae7bca4/events.out.tfevents.1759224840.1f0c1ae7bca4.211.0 (deflated 62%)
  adding: gpt2-finetuned/generation_config.json (deflated 24%)
  adding: gpt2-finetuned/training_args.bin (deflated 54%)
  adding: gpt2-finetuned/tokenizer_config.json (deflated 54%)
  adding: gpt2-finetuned/special_tokens_map.json (deflated 60%)
  adding: gpt2-finetuned/config.json (deflated 51%)
  adding: gpt2-finetuned/merges.txt (deflated 53%)
  adding: gpt2-finetuned/checkpoint-225/ (stored 0%)
  adding: gpt2-finetuned/checkpoint-225/generation_config.json (deflated 24%)
  adding: gpt2-finetuned/checkpoint-225/training_args.bin (deflated 54%)
  adding: gpt2-finetuned/checkpoint-225/tokenizer_config.json (deflated 54%)
  adding: gpt2-finetuned/checkpoint-225/special_tokens_map.json (deflated 60%)
  adding: gpt2-

In [ ]:
from google.colab import files
files.download("gpt2-finetuned.zip")
